# Data generating script for skeleton visualization
## Settings & tool function

In [1]:
import os
import json
import numpy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # 空间三维画图

# Setting for phoenix
# w = 210
# h = 260
# Setting for CSL
w = 720
h = 800
num_joints = 137
view_joints = numpy.array([0,1,2,3,4,5,6,7,8,9,12,15,16,17,18])
face_view_joints = numpy.arange(25,95)
lefhand_view_joints = numpy.arange(95,116)
righthand_view_joints = numpy.arange(116,137)
# view_joints = face_view_joints
view_joints = numpy.concatenate([view_joints,face_view_joints,lefhand_view_joints,righthand_view_joints],0)
# [
#     [4,3],
#     [6,7]
# ]
line_pairs = numpy.array(\
[
    [17,15],
    [15,0],
    [0,16],
    [16,18],
    [0,1],
    [2,1],
    [1,5],
    [1,8],
    [4,3],
    [3,2],
    [5,6],
    [6,7],
    [9,8],
    [8,12]
])
hand_pairs = numpy.array(\
[
    [0,1],
    [1,2],
    [2,3],
    [3,4],
    [0,5],
    [5,6],
    [6,7],
    [7,8],
    [0,9],
    [9,10],
    [10,11],
    [11,12],
    [0,13],
    [13,14],
    [14,15],
    [15,16],
    [0,17],
    [17,18],
    [18,19],
    [19,20]
])
lefthand_pairs = hand_pairs + 95
righthand_pairs = hand_pairs + 116
line_pairs = numpy.concatenate([line_pairs,lefthand_pairs,righthand_pairs],0)
# Colors
lightred = [0.93,0.60,0.60]
red = [0.83,0.18,0.18]
orange = [1.0,0.5,0.2]
blue = [0.0,0.66,0.88]
pink = [0.92,0.38,0.57]
purple = [0.70,0.53,1.0]
yellow = [1.0,0.93,0.35]

def read_json(file):
    skeletonDict = json.load(open(file,'r'))
    bodySkeleton = numpy.array(skeletonDict['Body']).squeeze()
    faceSkeleton = numpy.array(skeletonDict['Face']).squeeze()
    leftHandSkeleton = numpy.array(skeletonDict['Left hand']).squeeze()
    rightHandSkeleton = numpy.array(skeletonDict['Right hand']).squeeze()
    skeleton = numpy.concatenate([bodySkeleton, faceSkeleton, leftHandSkeleton, rightHandSkeleton],-2)
    if len(skeleton.shape)>2:
        skeleton = skeleton[0]
    return skeleton[:,:2]

def get_pairs(skeleton, line_pairs):
    pairs = []
    for pair in line_pairs:
        i = pair[0]
        j = pair[1]
        if (skeleton[i]==0).all() or (skeleton[j]==0).all():
            print(pair)
        else:
            pairs.append(pair)
    return numpy.array(pairs)

def get_colors():
    # Paint diffenrent color
    colors = []
    for i in range(num_joints):
        if i in [0,15,16,17,18]:
            color = red
        elif i in [1,8,9,12]:
            color = orange
        elif i in [2,3,4]:
            color = blue
        elif i in [5,6,7]:
            color = pink
        elif 25<=i and i<=94:
            color = lightred
        elif 95<=i and i<=115:
            color = purple
        elif 116<=i and i<=136:
            color = yellow
        colors.append(color)
    return numpy.array(colors)

def normalize(mat):
    # Shape of mat is: J x D
    # Actually it is mat_i
    max_x = numpy.max(mat[:,0])
    min_x = min(mat[:,0])
    max_y = numpy.max(mat[:,1])
    min_y = min(mat[:,1])
    center_x = (max_x+min_x)/2
    center_y = (max_y+min_y)/2
    mat = (mat-[center_x,center_y])/[(max_x-min_x)/2,(max_y-min_y)/2]
    # TEST
    # print("max_x: %.2f,min_x: %.2f,max_y: %.2f,min_y: %.2f"%(max_x,min_x,max_y,min_y))
    return mat

def augmentation(mat):
    # Shape of mat is T x J x D
    amp = 0.02
    d0,d1,d2 = mat.shape
    jitter = (numpy.random.rand(d0,d1,d2)-0.5)*amp*2
    mat = mat + jitter
    return mat

## Generate from json file (Unfold along time)

In [8]:
skeleton_folder = '/Users/liweijie/SLR/scripts/phoenix/30November_2011_Wednesday_tagesschau_default-18'
# skeleton_folder = '/Users/liweijie/SLR/scripts/csl/P42_s1_02_4._color'
file_list = os.listdir(skeleton_folder)
file_list.sort()
file_list = file_list[1:]
mat = []
point_indices = []
line_indices = []
for i,file in enumerate(file_list[::1]):
    file_path = os.path.join(skeleton_folder,file)
    skeleton = read_json(file_path)
    skeleton = (skeleton-numpy.array([w/2,h/2]))*2/numpy.array([w,h])
    t = 4*10*(i/len(file_list))*numpy.ones([skeleton.shape[0],1])
    colors = get_colors()
    skeleton_t = numpy.concatenate([skeleton,t,colors],1)
    point_indices_t = i*num_joints+numpy.array(view_joints)
    line_indices_t = i*num_joints+get_pairs(skeleton,line_pairs)
    # Append
    mat.append(skeleton_t)
    point_indices.append(point_indices_t)
    line_indices.append(line_indices_t)
mat = numpy.stack(mat,0)
point_indices = numpy.stack(point_indices,0)
line_indices = numpy.stack(line_indices,0)

# Save as byte data
mat.astype(numpy.float32).tofile('/Users/liweijie/projects/visualize-skeleton/skeleton.data')
point_indices.astype(numpy.uint32).tofile('/Users/liweijie/projects/visualize-skeleton/point_indices.data')
line_indices.astype(numpy.uint32).tofile('/Users/liweijie/projects/visualize-skeleton/line_indices.data')

## Generate from json file (Unfold along horizontal)
- from CSL dataset

In [6]:
skeleton_folder = '/Users/liweijie/SLR/scripts/csl/P42_s1_02_4._color'
file_list = os.listdir(skeleton_folder)
file_list.sort()
mat = []
point_indices = []
line_indices = []
for i,file in enumerate(file_list[1::2]):
    file_path = os.path.join(skeleton_folder,file)
    skeleton = read_json(file_path)
    skeleton = (skeleton-numpy.array([w/2,h/2]))*2/numpy.array([w,h])
    skeleton = skeleton + [0.8*i,0]
    t = numpy.zeros([skeleton.shape[0],1])
    colors = get_colors()
    skeleton_t = numpy.concatenate([skeleton,t,colors],1)
    point_indices_t = i*num_joints+numpy.array(view_joints)
    line_indices_t = i*num_joints+get_pairs(skeleton,line_pairs)
    # Append
    mat.append(skeleton_t)
    point_indices.append(point_indices_t)
    line_indices.append(line_indices_t)
mat = numpy.stack(mat,0)
point_indices = numpy.stack(point_indices,0)
line_indices = numpy.stack(line_indices,0)

# Save as byte data
mat.astype(numpy.float32).tofile('/Users/liweijie/projects/visualize-skeleton/skeleton.data')
point_indices.astype(numpy.uint32).tofile('/Users/liweijie/projects/visualize-skeleton/point_indices.data')
line_indices.astype(numpy.uint32).tofile('/Users/liweijie/projects/visualize-skeleton/line_indices.data')

## Generate from npy

In [ ]:
import numpy
npy_file = 'vae_generate_ISL/input_000000.npy'
# Shape of data is N x L(32) x J x D
data = numpy.load(npy_file)
data = data[0]
mat = []
point_indices = []
line_indices = []
for i,skeleton in enumerate(data):
    skeleton = (skeleton-numpy.array([w/2,h/2]))*2/numpy.array([w,h])
    t = 2*10*(i/len(data))*numpy.ones([skeleton.shape[0],1])
    colors = get_colors()
    skeleton_t = numpy.concatenate([skeleton,t,colors],1)
    point_indices_t = i*num_joints+numpy.array(view_joints)
    line_indices_t = i*num_joints+line_pairs
    # Append
    mat.append(skeleton_t)
    point_indices.append(point_indices_t)
    line_indices.append(line_indices_t)
mat = numpy.stack(mat,0)
point_indices = numpy.stack(point_indices,0)
line_indices = numpy.stack(line_indices,0)

# Save as byte data
mat.astype(numpy.float32).tofile('/Users/liweijie/projects/visualize-skeleton/skeleton.data')
point_indices.astype(numpy.uint32).tofile('/Users/liweijie/projects/visualize-skeleton/point_indices.data')
line_indices.astype(numpy.uint32).tofile('/Users/liweijie/projects/visualize-skeleton/line_indices.data')

## Visualize input & recons simultaneously to compare

In [ ]:
import numpy
id = 1000
# generate from npy
npy_file = 'vae_generate_ISL/input_%06d.npy'%id
# Shape of data is N x L(32) x J x D
data = numpy.load(npy_file)
data = data[0]
mat = []
point_indices = []
line_indices = []
for i,skeleton in enumerate(data):
    skeleton = (skeleton-numpy.array([w/2,h/2]))*2/numpy.array([w,h])
    t = 2*10*(i/len(data))*numpy.ones([skeleton.shape[0],1])
    colors = get_colors()
    skeleton_t = numpy.concatenate([skeleton,t,colors],1)
    point_indices_t = i*num_joints+numpy.array(view_joints)
    line_indices_t = i*num_joints+line_pairs
    # Append
    mat.append(skeleton_t)
    point_indices.append(point_indices_t)
    line_indices.append(line_indices_t)

bias = 32*num_joints
# read recons from npy
npy_file = 'vae_generate_ISL/recons_%06d.npy'%id
data = numpy.load(npy_file)
data = data[0]
for i,skeleton in enumerate(data):
    skeleton = (skeleton-numpy.array([w/2,h/2]))*2/numpy.array([w,h])
    skeleton = skeleton + numpy.array([1.5,0])
    t = 2*10*(i/len(data))*numpy.ones([skeleton.shape[0],1])
    colors = get_colors()
    skeleton_t = numpy.concatenate([skeleton,t,colors],1)
    point_indices_t = i*num_joints+numpy.array(view_joints)+bias
    line_indices_t = i*num_joints+line_pairs+bias
    # Append
    mat.append(skeleton_t)
    point_indices.append(point_indices_t)
    line_indices.append(line_indices_t)

mat = numpy.stack(mat,0)
point_indices = numpy.stack(point_indices,0)
line_indices = numpy.stack(line_indices,0)
# Save as byte data
mat.astype(numpy.float32).tofile('/Users/liweijie/projects/visualize-skeleton/skeleton.data')
point_indices.astype(numpy.uint32).tofile('/Users/liweijie/projects/visualize-skeleton/point_indices.data')
line_indices.astype(numpy.uint32).tofile('/Users/liweijie/projects/visualize-skeleton/line_indices.data')

## Visualize data & augmentation simultaneously to compare

In [7]:
import numpy
id = 1000
# generate from npy
npy_file = 'isl/000000/P01_01_00_0_color.npy'
# Shape of data is T x J x D
data = numpy.load(npy_file)
mat = []
point_indices = []
line_indices = []
for i in range(len(data)):
    data[i] = normalize(data[i])
data_aug = augmentation(data)
scale = 4
for i,skeleton in enumerate(data[::scale]):
    t = 2*10*(i/len(data))*numpy.ones([skeleton.shape[0],1])
    colors = get_colors()
    skeleton_t = numpy.concatenate([skeleton,t,colors],1)
    point_indices_t = i*num_joints+numpy.array(view_joints)
    line_indices_t = i*num_joints+line_pairs
    # Append
    mat.append(skeleton_t)
    point_indices.append(point_indices_t)
    line_indices.append(line_indices_t)

bias = len(data[::scale])*num_joints
# generate for data aug
for i,skeleton in enumerate(data_aug[::scale]):
    skeleton = skeleton + numpy.array([1.5,0])
    t = 2*10*(i/len(data))*numpy.ones([skeleton.shape[0],1])
    colors = get_colors()
    skeleton_t = numpy.concatenate([skeleton,t,colors],1)
    point_indices_t = i*num_joints+numpy.array(view_joints)+bias
    line_indices_t = i*num_joints+line_pairs+bias
    # Append
    mat.append(skeleton_t)
    point_indices.append(point_indices_t)
    line_indices.append(line_indices_t)

mat = numpy.stack(mat,0)
point_indices = numpy.stack(point_indices,0)
line_indices = numpy.stack(line_indices,0)
# Save as byte data
mat.astype(numpy.float32).tofile('/Users/liweijie/projects/visualize-skeleton/skeleton.data')
point_indices.astype(numpy.uint32).tofile('/Users/liweijie/projects/visualize-skeleton/point_indices.data')
line_indices.astype(numpy.uint32).tofile('/Users/liweijie/projects/visualize-skeleton/line_indices.data')